In [4]:
import os
import json
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from matplotlib import pyplot as plt

In [ ]:
# Load the data

train_data = pd.read_json("train_rectangle.json", orient="records")
test_data = pd.read_json("test_rectangle.json", orient="records")

x_train = train_data.iloc[:, 0:118]
y_train = train_data[118]

x_test = test_data.iloc[:, 0:118]
y_test = test_data[118]

print(x_train)
print(y_train)

In [5]:
# Define create and train model functions

def create_model(my_learning_rate):
    model = tf.keras.models.Sequential()
    # hidden layers
    model.add(tf.keras.layers.Input(
        shape=[None, None, None, 118], dtype=tf.int64))

    for i in range(0, 5):
        model.add(tf.keras.layers.Dense(units=22, activation="relu"))

    model.add(tf.keras.layers.Dense(units=4, activation='softmax'))
    # compile
    model.compile(optimizer=tf.keras.optimizers.Adamax(learning_rate=my_learning_rate),
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model


def train_model(model, train_features, train_label, epochs,
                batch_size=None, validation_split=0.1):
    history = model.fit(x=train_features, y=train_label, batch_size=batch_size,
                        epochs=epochs, shuffle=True,
                        validation_split=validation_split)

    epochs = history.epoch
    history = pd.DataFrame(history.history)

    return epochs, history

In [ ]:
# Create and train the model

learning_rate = 0.003
epochs = 100
batch_size = 16
val_split = 0.9

my_model = create_model(learning_rate)

epochs, hist = train_model(my_model, x_train, y_train, epochs, batch_size)

In [ ]:
def plot_model(epochs, hist, list_of_metrics):

    plt.figure()
    plt.xlabel("Epoch")
    plt.ylabel("Value")

    for m in list_of_metrics:
        x = hist[m]
        plt.plot(epochs[1:], x[1:], label=m)
    plt.legend()

In [ ]:
# Plot the metric

list_of_metrics_to_plot = ["accuracy"]
plot_model(epochs, hist, list_of_metrics_to_plot)

my_model.evaluate(x=x_test, y=y_test, batch_size=batch_size)


print([learning_rate,
       len(epochs),
       batch_size,
       val_split])
print(my_model.summary())